In [14]:
import torch
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, CODECONTESTS_DPO_CONFIG, SMOL_LM_135M

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
cfg = CODECONTESTS_DPO_CONFIG
cfg.notebook_mode = True
cfg.model_id_or_path = SMOL_LM_135M
wrapper = TrainerWrapper(cfg)
print(cfg.input_dataset_path)

jondurbin/py-dpo-v0.1


In [8]:
wrapper.init_model()

2025-01-12 14:25:31.012 | INFO     | trl_wrapper.trainer_wrapper:init_model:216 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct with attn_impl: sdpa


In [16]:
wrapper.init_data_module()

2025-01-12 14:29:04.609 | INFO     | model.utils:__init__:110 - Cache dir: ../dataset_caches/conversation_data_module
2025-01-12 14:29:04.609 | INFO     | model.utils:setup:136 - Loading dataset for stage fit
2025-01-12 14:29:04.610 | INFO     | model.utils:setup:139 - Processing dataset for stage fit, workers: 1, cache dir ../dataset_caches/conversation_data_module, using cache: False
2025-01-12 14:29:04.610 | INFO     | model.utils:load_dataset:118 - Loading dataset


FileNotFoundError: Unable to find '/Users/brianfitzgerald/smolmodels/notebooks/../jondurbin/py-dpo-v0.1'

In [ ]:
# Tokenizer dev
first_batch = wrapper.data_module.train_dataset[0]
tokenized = wrapper.data_module._tokenize_conversation(first_batch['conversation'])
for k in tokenized.keys():
    tokenized[k] = torch.as_tensor(tokenized[k], dtype=torch.long)
print(tokenized['labels'].tolist())
wrapper.data_module.visualize_sample(tokenized)

In [ ]:
# mock_labels = first_batch["labels"].clone().detach()
# mock_labels = torch.where(mock_labels == -100, wrapper.data_module.tokenizer.pad_token_id, mock_labels)
# mock_ids = first_batch["input_ids"].clone().detach()
for batch in wrapper.data_module.val_dataloader():
    print(batch["input_ids"].shape)
    print(batch["labels"].shape)

In [ ]:
wrapper.init_trainer()

In [ ]:
wrapper.tokenizer

In [ ]:
wrapper.train()